In [1]:
import pandas as pd
import numpy as np 
import re

In [2]:
df = pd.read_csv("./from_skip_files/orders_3.5mil.csv")
item_rf = pd.read_csv("./derived_files/restaurant_profiles.csv")
rf_c = pd.read_csv('./derived_files/Cuisine_Restaraunt.csv')

In [3]:
print(df['restaurant_short_name'].nunique())
print(item_rf['short_name'].nunique())
print(df['restaurant_short_name'].str.contains("DNU").sum())
# print(df['restaurant_short_name'].str.contains("License #").sum())
def remove_after_dash(s):
    return s.split("- Li")[0].strip()

df['short_name_clean'] = df['restaurant_short_name'].apply(remove_after_dash)
df['short_name_clean'] = df['short_name_clean'].str.lower()

print(df['short_name_clean'].nunique())

list1 = df['short_name_clean'].unique()
list2 = item_rf['short_name'].unique()
opposite = set(list1) - set(list2)
opposite2 = set(list2) - set(list1)
print(len(opposite))
print(len(opposite2))

3113
2354
38
2808
702
248


In [4]:
ethnic_or_cultural_items = [
    "Convenience",
    "Indian",
    "Italian",
    "Chinese",
    "Vietnamese",
    "Japanese",
    "Mediterranean",
    "Middle Eastern",
    "Mexican",
    "Korean",
    "Thai",
    "French",
    "African",
    "Latin American",
    "Ethiopian",
    "Caribbean",
    "Filipino",
    "Spanish",
    "German",
]

In [5]:
# Map of cuisine type to keywords that fit under that cuisine
cuisine_map = {
    "Chicken" : ["chicken"],
    "Fries" : ["fries"],
    "Beef" : ["beef"],
    "Pork" : ["pork", "bacon", "pepperoni"],
    "Rice" : ["rice"],
    "Lamb" : ["lamb"],
    "Vegetarian" : ["vegan", "vegetarian", "veggie", "beyond meat"],
    "Sandwiches & Subs" : ["sandwich", "sub", "wrap", "blt"],
    "Desserts" : ["blizzard", "ice cream", "frozen", "dessert", "chocolate", "drizzle", "desserts", "milkshake", "candy", 
                  "candies", "sundae", "oreo", "skor", "brownie", "shake", "tiramisu", "timbits", "ben and jerry", 
                  "cheesecake", "cookie"],
    "Canadian" : ["canadian", "canadien", "alberta","poutine"],
    "Fast food" : ["fast food","combo", "meal"],
    "Burgers" : ["burger", "patty", "mcdouble", "big mac", "quarter pounder"],
    "Seafood" : ["fish", "seafood", "shrimp", "crab", "lobster", "prawn", "seaweed", 
                 "salmon", "tuna", "poke", "calamari", "squid", "fish and chips"],
    "Healthy" : ["organic", "health", "protein", "salad", "fresh", "tofu",  "fruit", "water", "vegetable", "smoothie", "parfait"],
    "Pizza" : ["pizza"],
    "Breakfast & Brunch" : ["egg", "toast", "benedict", "breakfast", "brunch", "cereal", "pancake", "waffle", "hash brown"],
    "Coffee/Tea" : ["coffee", "tea", "americano", "cappuccino", "latte", "cafe", "chai", "london fog"],
    "Alcohol" : ["beer", "wine", "liquor", "budweiser", "bud light", "spirits", "corona", "stella artois", 
                 "michelob ultra", "mike's hard", "labatt", "sauvignon", "smirnoff", "vodka", "whisky", "cognac",
                 "white claw", "pinot noir"],
    "Noodles" : ["noodle", "vermicelli"],
    "Pub food" : ["wing", "onion ring", "wedge", "mac & cheese", "mac and cheese", "gravy", "mashed potato", "breadsticks"], 
    "Indian" : ["indian", "naan", "nan", "samosa", "masala", "aloo", "paneer", "biryani", "tandoori", "roti", "tikka"],
    "Italian" : ["italian", "pasta", "spaghetti", "penne", "fettuccini", "lasagna", "lasagne", "linguini", "ravioli", "tortellini", "meatball", "canoli"],
    "Bakery" : ["danish", "cake", "bun", "donut", "muffin","bagel", "doughnut", "pie", "scone", "rolls", "loaf"],
    "Barbecue" : ["barbecue", "bbq", "grill", "buffalo"],
    "Chinese" : ["chinese", "china", "hot pot", "wonton", "cantonese", "mein", "gyoza"],
    "Vietnamese" : ["vietnamese", "pho", "viet", "bun cha","ca kho to"],
    "Japanese" : ["japanese","japan", "ramen", "sashimi", "teriyaki", "katsu", "tempura", "edamame", "bento", "takoyaki"],
    "Tacos": ["taco"],
    "Sushi" : ["sushi"],
    "Mediterranean" : ["mediterranean","pita", "damascus", "greek", "greece", "briam","taramasalata", "opa"],
    "Hot Dogs & Sausages" : ["hot dogs","sausage", "weiner"],
    "Middle Eastern" : ["middle eastern","falafel", "hummus","shawarma", "baklava", "donair", "tzatziki"],
    "Convenience" : ["convenience","pre-made","grocery", "slurpee"],
    "Mexican" : ["mexican", "chilaquiles", "burrito", "nacho", "quesadilla", "queso", "taquito", "salsa"],
    "Steakhouse" : ["steakhouse", "steak"],
    "Halal" : ["halal", "zabiha"],
    "Korean" : ["korean", "kimchi", "bulgogi","bibimbap","tteokbokki","jjambbong","doenjang"],
    "Thai" : ["thai", "tom yum goong", "green curry"],
    "Soup" : ["soup"],
    "Gluten Free" : ["gluten free", "no gluten"],
    "Popcorn": ["popcorn"], 
    "Pet Food" : ["pet", "dog", "cat"],
    "Bubble Tea" : ["bubble tea", "boba", "milk tea", "taro milk"],
    "French" : ["french", "francais", "crepe", "foie gras","coq au vin","cassoulet","baguette","croissant","gougeres","cajun & creole", "creole"],
    "African" : ["african","pap en vleis", "shisa nyama","bunny chow","koshari"],
    "Latin American" : ["latin","asado","saltena","feijoada","empanada","bandeja paisa",
                        "gallo pinto","ropa vieja","mangu","encebollado", "pupusas","pepian","peruvian"],
    "Haute Cuisine" : ["haute", "high class", "expensive", "champagne"],
    "Ethiopian" : ["ethiopian","tibs","kitfo","beyainatu","fuul"],
    "Caribbean" : ["caribbean", "jamaica","barbados","bahamas"],
    "Filipino" : ["filipino","adobo","lechon","sisig","bulalo"],
    "Spanish" : ["spanish","paella valenciana","patatas bravas","gazpacho","pimientos de padron","jamon","tapas","churro"],
    "Butcher" : ["raw", "butcher","delicatessen"],
    "Kosher" : ["kosher", 'kashrut', 'jewish'],
    "German" : ["german","schnitzel","rouladen","eintopf","sauerbraten"]
}
print(len(list(cuisine_map.keys())))

54


In [6]:
grouped = df.groupby('short_name_clean')
restaurant_order_profile_list = []
def process_group(group):
    restaurant_order_profile = dict(zip(cuisine_map.keys(), [0] * 54))
    total_filtered = 0
    for index, row in group.iterrows():
        items = row['item_list'].split(", ")
        any_items_found = False
        for item in items:
            lower_item = item.lower()
            cuisine_identified_item = False
            for key in cuisine_map:
                for cuisine in cuisine_map[key]:
                    if cuisine in lower_item:    
                        if key in ethnic_or_cultural_items:
                            cuisine_identified_item = True  
                        restaurant_order_profile[key] += 1
                        total_filtered += 1 
                        any_items_found = True
                        break
            if not cuisine_identified_item:
                restaraunt_cuisines_df = rf_c[rf_c['short_name'] == row['short_name_clean']]['cussine_list']
                if restaraunt_cuisines_df.shape[0] > 0:
                    restaraunt_cuisines_list = restaraunt_cuisines_df.str.split(", ")
                    restaraunt_cuisines = restaraunt_cuisines_list.iloc[0]
                    for cuisine in restaraunt_cuisines:
                        restaurant_order_profile[cuisine] +=1 
                        total_filtered += 1 
    # Error checking
    # print( total_filtered)
    map_total_filtered = 0
    for value in restaurant_order_profile.values():
        map_total_filtered += value
    # Normalizing
    if total_filtered == map_total_filtered and total_filtered != 0:
        for key in restaurant_order_profile:
            restaurant_order_profile[key] = restaurant_order_profile[key]/total_filtered
        restaurant_order_profile['short_name'] = group.name
        restaurant_order_profile['number_of_orders'] = group.shape[0]
        restaurant_order_profile_list.append(restaurant_order_profile)
    else:
      print("missed restauraunt: ", group.name)
      print(group)
    if len(restaurant_order_profile_list) % 300 == 0:
        print(len(restaurant_order_profile_list)/2800)

grouped.apply(process_group)

missed restauraunt:  aspen wine & spirits
                                           id  \
973163   9ca835b9-98de-448f-a6fb-8e541c7193b1   
973273   271c8777-88c5-45a0-8301-dadb31a62590   
973330   f4b8a9e2-f945-491d-845a-0d0d947a04ed   
2439623  20cc1856-9ea2-4113-8df1-a7d801ebf2e0   

                                  customer_id             created_time  \
973163   08b21fd6-3ce1-4d93-9053-6fff3d37a20e  2022-11-12 01:04:36 UTC   
973273   08b21fd6-3ce1-4d93-9053-6fff3d37a20e  2022-11-10 04:16:24 UTC   
973330   08b21fd6-3ce1-4d93-9053-6fff3d37a20e  2022-11-08 02:53:48 UTC   
2439623  158a6f92-654b-4d99-b3d6-4aa000d5db74  2022-11-07 04:59:17 UTC   

                                restaurant_id restaurant_short_name  \
973163   a9193e4d-4a68-4c37-9420-b63ea45ff84e  Aspen Wine & Spirits   
973273   a9193e4d-4a68-4c37-9420-b63ea45ff84e  Aspen Wine & Spirits   
973330   a9193e4d-4a68-4c37-9420-b63ea45ff84e  Aspen Wine & Spirits   
2439623  a9193e4d-4a68-4c37-9420-b63ea45ff84e  Aspen Wine

missed restauraunt:  bumbu indonesian grill
                                          id  \
403046  185191cd-68ed-4fa9-a91b-ab0148c5f3ee   

                                 customer_id             created_time  \
403046  038f60e2-4009-481b-b2d4-10b20976e424  2020-01-18 00:23:16 UTC   

                               restaurant_id   restaurant_short_name  \
403046  39aed725-6a9f-419a-a3a0-d57d3e6b51c9  Bumbu Indonesian Grill   

        price_details_delivery_fee  price_details_tip  \
403046                         0.0               3.75   

        price_details_restaurant_subsidy  price_details_skip_subsidy  \
403046                              2.99                        2.76   

        delay_requested  restaurant_food_prep_minutes  alcohol_delivery  \
403046              NaN                            10                 0   

                     item_list        short_name_clean  
403046  Mie Goreng, Mie Goreng  bumbu indonesian grill  
missed restauraunt:  burwood distillery
  

missed restauraunt:  ginger garlic south indian restaurant
                                           id  \
2091988  8be1e148-228e-4837-aff9-6ca94c8f3300   

                                  customer_id             created_time  \
2091988  1292f146-2ac7-4f90-a20c-34e23196f683  2023-01-15 04:04:17 UTC   

                                restaurant_id  \
2091988  fe22f161-074e-48f1-99e7-9cdae612de9e   

                         restaurant_short_name  price_details_delivery_fee  \
2091988  Ginger Garlic South Indian Restaurant                        6.99   

         price_details_tip  price_details_restaurant_subsidy  \
2091988                NaN                               0.0   

         price_details_skip_subsidy  delay_requested  \
2091988                        0.66              2.0   

         restaurant_food_prep_minutes  alcohol_delivery  \
2091988                            15                 0   

                         item_list                       short_name_clean  


missed restauraunt:  minhas liquor store
                                           id  \
800689   78500b86-a432-4415-a683-d0d6e6d51a93   
1378342  3bfd3700-e45e-4a15-b4a5-51126e0b7e9f   
2011866  8fe97e55-2d7f-4393-b5f0-cde55140fc9e   

                                  customer_id             created_time  \
800689   071e864c-61b5-4d1c-bf5d-404896a41431  2022-08-13 04:25:53 UTC   
1378342  0c5490dd-b235-410a-86a5-4ef4a163f99b  2021-10-28 20:42:31 UTC   
2011866  11e1f340-d9b1-4dae-ae5b-d203664fe52c  2022-12-01 00:54:38 UTC   

                                restaurant_id restaurant_short_name  \
800689   df654e88-efe0-486d-b81f-d30d3824a87b   Minhas Liquor Store   
1378342  df654e88-efe0-486d-b81f-d30d3824a87b   Minhas Liquor Store   
2011866  df654e88-efe0-486d-b81f-d30d3824a87b   Minhas Liquor Store   

         price_details_delivery_fee  price_details_tip  \
800689                         5.25               2.00   
1378342                        2.99               4.05   
201186

missed restauraunt:  ray's pies n' calzones
                                           id  \
1916985  c7186dd0-97d1-4977-9794-b4f41eee2afa   

                                  customer_id             created_time  \
1916985  11085735-a3c7-4979-88d2-c000cf15b296  2023-01-11 03:23:29 UTC   

                                restaurant_id   restaurant_short_name  \
1916985  aa740fad-db63-4aee-8edd-de6e38d1d42b  Ray's Pies N' Calzones   

         price_details_delivery_fee  price_details_tip  \
1916985                         0.0               5.08   

         price_details_restaurant_subsidy  price_details_skip_subsidy  \
1916985                              2.99                        2.76   

         delay_requested  restaurant_food_prep_minutes  alcohol_delivery  \
1916985              5.0                            15                 0   

                                                 item_list  \
1916985  Cheese Calzone, Build Your Own Calzone, Mozza ...   

               shor

missed restauraunt:  the o.t. brewing company
                                           id  \
1794727  219f653b-8d21-4edf-bd52-3d7f03d5ad32   

                                  customer_id             created_time  \
1794727  1002044d-5550-4119-8f91-1a55962e73d7  2020-03-27 20:42:13 UTC   

                                restaurant_id  \
1794727  45186721-9b95-4126-b45b-702ee4b58fbc   

                              restaurant_short_name  \
1794727  The O.T. Brewing Company - License #776989   

         price_details_delivery_fee  price_details_tip  \
1794727                        4.99               4.65   

         price_details_restaurant_subsidy  price_details_skip_subsidy  \
1794727                               0.0                        0.76   

         delay_requested  restaurant_food_prep_minutes  alcohol_delivery  \
1794727              0.0                            15                 1   

                                                 item_list  \
1794727  5 Hundie

missed restauraunt:  uzu taiyaki
                                           id  \
234210   ca8ff28e-194d-44f8-ac41-8a96d95f01a2   
589855   4d442189-c572-483f-87ca-30f7a00343e5   
940374   4f01e52c-ef2b-482f-bf88-a42d116be6b5   
2555855  087cdb2a-2f53-4249-9fef-31f8d0fc96d8   
3043558  0ecf6151-b964-422b-813a-03203adf31b9   
3226897  8a923ef4-7b95-4984-8732-9540ed663f3f   

                                  customer_id             created_time  \
234210   0218cac3-160a-4f86-999f-0c1fcf9aced8  2021-09-11 21:15:29 UTC   
589855   0530b8b6-5936-4bec-bc2c-385b1489a5c7  2021-10-02 00:09:03 UTC   
940374   0869ec22-1736-4eb7-bfb0-8fdf96d4df98  2021-02-20 02:00:06 UTC   
2555855  16856f0e-f173-4f32-b5bb-5373e33c2909  2021-08-13 23:52:37 UTC   
3043558  1aeedfae-ef1f-4f5a-8897-c2461721d2e4  2021-04-03 21:14:51 UTC   
3226897  1c86f021-d03f-4d2e-a45b-b21dbf33ff00  2021-07-15 00:56:37 UTC   

                                restaurant_id restaurant_short_name  \
234210   b473bc39-2db9-4c5a-a100-

""


In [7]:
# restaurant_order_profile_df = pd.DataFrame(restaurant_order_profile_list)
# restaurant_order_profile_df = pd.read_csv('restaurant_order_profile.csv')
# restaurant_order_profile_df = restaurant_order_profile_df.drop("Unnamed: 0", axis=1)
# print(df['restaurant_id'].nunique())
# print(restaurant_order_profile_df['restaurant_id'].nunique())

In [8]:
restaurant_order_profile_df = pd.DataFrame(restaurant_order_profile_list)
restaurant_order_profile_df = restaurant_order_profile_df.reindex(columns=['short_name', 'number_of_orders'] + list(restaurant_order_profile_df.columns.drop(['short_name', 'number_of_orders'])))
cols = restaurant_order_profile_df.columns.tolist()
cols = cols[:-54] + sorted(cols[-54:])
restaurant_order_profile_df = restaurant_order_profile_df.reindex(columns=cols)
print(restaurant_order_profile_df)

                     short_name  number_of_orders  African   Alcohol  \
0                         1 pot                12      0.0  0.000000   
1          1410 world bier haus               107      0.0  0.000000   
2          1600 world bier haus               599      0.0  0.000000   
3      17th ave liquor boutique              6288      0.0  0.616227   
4     17th ave. liquor boutique               236      0.0  0.709434   
...                         ...               ...      ...       ...   
2765                 zcrew cafe               525      0.0  0.000000   
2766                zen 8 grill                37      0.0  0.000000   
2767         zero issue brewing                 6      0.0  0.000000   
2768        zimidara restaurant                84      0.0  0.000000   
2769                      zushi                10      0.0  0.000000   

        Bakery  Barbecue      Beef  Breakfast & Brunch  Bubble Tea   Burgers  \
0     0.118644  0.076271  0.059322            0.050847 

In [9]:
restaurant_order_profile_df.to_csv('restaurant_order_profile.csv', index=False)

In [10]:
def calculate_weight(num_orders, upper_bound):
    weight = 0
    if(num_orders >= 50 and num_orders <= 200):
        weight = (num_orders**(0.07652714627))-1 # This num comes from making weight 0.5 at 200 with exponents
        # ln(1.5)/ln(200)
    elif(num_orders > 200 and num_orders <= 600):
        weight = 0.001667*(num_orders)+0.1666 # Slope that becomes 1.0 at 600 orders
    elif(num_orders < 50):
        weight = 0
    else:
        weight = 1
    # multiply weights by upper bound    
    if(weight>upper_bound):
        weight = upper_bound
    if(weight < 0):
        weight = 0
    return weight

In [11]:
def combine_order_and_item_profile(order, item, weight):
    new_vector = (weight)*(order.values)+(1-weight)*(item.values)
    return new_vector

In [12]:
print(item_rf.shape)
print(restaurant_order_profile_df.shape)

(2354, 55)
(2770, 56)


In [13]:
# Find restaraunt overlap between orders and items
print(item_rf.shape)
print(restaurant_order_profile_df.shape)
overlap = restaurant_order_profile_df['short_name'].isin(item_rf['short_name'])
ro_filtered = restaurant_order_profile_df[overlap]
print(ro_filtered.shape)

# For restaraunts overlap between orders and items 
rows = []
upper_bound_order_weighting = 0.8
for index, ord_row in ro_filtered.iterrows():
    num_orders = ord_row['number_of_orders']
    r_id = ord_row['short_name']
    # Get weight of combination
    weight = calculate_weight(num_orders, upper_bound_order_weighting)
#     print(num_orders, " -- ", weight)
    item_row = item_rf[item_rf['short_name'] == r_id]
    item_row = item_row.squeeze()
    item_row = item_row.drop('short_name')
    ord_row = ord_row.drop("number_of_orders")
    ord_row = ord_row.drop('short_name')
    new_vector = combine_order_and_item_profile(ord_row, item_row, weight)
    new_vector = np.insert(new_vector, 0, r_id)
    rows.append(new_vector)

(2354, 55)
(2770, 56)
(2087, 56)


In [14]:
column_names = item_rf.columns.to_list()
combined_df = pd.DataFrame(rows, columns=column_names)
print(combined_df.shape)

(2087, 55)


In [15]:
# Find missing restaraunts (restaurants not found in any orders)
difference = ~item_rf['short_name'].isin(restaurant_order_profile_df['short_name'])
no_order_restauraunts = item_rf[difference]

In [16]:
result_df = pd.concat([combined_df, no_order_restauraunts])
print(result_df.shape)
print(result_df['short_name'].nunique())
# Make sure each row adds to 1 
for index, row in result_df.iterrows():
    if row[1:].sum() > 1.05 or row[1:].sum() < 0.95:
      print(row[1:].sum())
      print("Something is wrong at index: ", index)


(2354, 55)
2354


In [17]:
result_df.to_csv('./derived_files/combined_restaurant_order_profile.csv', index=False)